# <CENTER>Tweet sentiment analysis</CENTER>

## TODO

- (Embedding tweet instead of google news)
- Add OLE on embedding (+2 into dim)
- Add EV on embedding (+11 into dim)
- 
- Cross validation
- Comment vérifier l'overfitting?
- Jouer avec hyperparametres (dropout, fonction activation, nb de neuronnes, nb de layers, nb d'epoche / batch, optimisation, architecture, 

## 0. Preparation

### Imports

In [1]:
# Python core and 3rd-party libraries
import numpy as np
import pandas as pd
import psutil # RAM monitoring

from gensim.models import KeyedVectors

from sklearn.model_selection import train_test_split

from keras.layers import Dense, Dropout, GRU, LSTM
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

# Our custom preprocessing module
from preprocessing import data_preprocessing, data_preprocessing_test

Using TensorFlow backend.


In [2]:
#from keras.layers import Bidirectional, Flatten, Input 

#import matplotlib as mpl
#from preprocessing import standardization, data_preprocessing
#mpl.use('TkAgg')  # or whatever other backend that you want

### Settings

In [3]:
np.random.seed(7) # random state

CORPUS_TRAIN_3 = "data/tweet/data_train_3.csv" # training part of the corpus for the model with 3 output classes
CORPUS_TEST_3 = "data/tweet/data_test_3.csv" # testing part of the corpus for the model with 3 output classes
CORPUS_TRAIN_7 = "data/tweet/data_train_7.csv" # training part of the corpus for the model with 7 output classes

# FIXME: Currently, not available!
CORPUS_TEST_7 = CORPUS_TEST_3  # testing part of the corpus for the model with 7 output classes
#CORPUS_TEST_7 = "data/task_A/data_test_7.csv"

EMBEDDING_W2C_FILE = "data/embedding/GoogleNews-vectors-negative300.bin" # Google word2vec's file (~3.5GB)
EMBEDDING_MATRIX_FILE = "data/embedding/embedding_matrix.csv" # the embedding matrix exported to CSV (~200MB)
BASE_EMBEDDING_DIM = 300 # Embedding (word2vec) dimension

# FIXME
EXTRA_EMBEDDING_DIM = 0 # Add extra dimensions for polarities, etc
EMBEDDING_DIM = BASE_EMBEDDING_DIM + EXTRA_EMBEDDING_DIM # Total embedding dimensions

# Required memory (in GB) to run this notebook smoothly
REQUIRED_MEMORY = 6

## 1. Preprocessing

### Preprocess the tweets

Example:

"Won the match #getin . Plus\u002c tomorrow is a very busy day\u002c with Awareness Day\u2019s and debates. Gulp. Debates..." <BR>
-> "win match plus tomorrow busy day awareness day debate gulp debate ..."

**FIXME** for \*\_train_7: data_preprocessing(CORPUS_TRAIN_7, **'train'**) **???**

In [4]:
tweets_train_3, sentiments_train_3 = data_preprocessing(CORPUS_TRAIN_3, 'train')
tweets_train_7, sentiments_train_7 = data_preprocessing(CORPUS_TRAIN_7, 'test')

#FIXME 
#data = pd.read_csv("/home/abdou/Documents/datastories-semeval2017-task4-master/dataset/Subtask_A/gold/2018-Valence-oc-En-test-gold.txt", sep='\t', encoding='utf-8')
#tweets_test_7 = data_preprocessing_test(data)
tweets_test_7 = tweets_train_7

### Tokenize the tweets and get the word index

In [5]:
tokenizer = Tokenizer(filters=' ')
all_tweets = tweets_train_3.append(tweets_train_7) # append both Pandas series
tokenizer.fit_on_texts(all_tweets)
word_index = tokenizer.word_index # a dictionary {word: index}
len(word_index)

37784

### Create sequences

Example: "win match plus tomorrow busy day awareness day debate gulp debate ..." -> [43, 234, 769, 2, 1054, 6, 2896, 6, 424, 12301, 424, 1]

In [6]:
sequences_train_3 = tokenizer.texts_to_sequences(tweets_train_3)
sequences_train_7 = tokenizer.texts_to_sequences(tweets_train_7)
sequences_test_7 = tokenizer.texts_to_sequences(tweets_test_7)

### Calculate the max tweet sequence and pad data

In [7]:
MAX_SEQUENCE_LENGTH = max(len(elt) for elt in sequences_train_3 + sequences_train_7)
MAX_SEQUENCE_LENGTH

32

In [8]:
data_train_3 = pad_sequences(sequences_train_3, maxlen=MAX_SEQUENCE_LENGTH)
data_train_7 = pad_sequences(sequences_train_7, maxlen=MAX_SEQUENCE_LENGTH)
data_test_7 = pad_sequences(sequences_test_7, maxlen=MAX_SEQUENCE_LENGTH)

### Create the embedding matrix

In [9]:
psutil.virtual_memory()

svmem(total=8241569792, available=3070816256, percent=62.7, used=4467982336, free=1106837504, active=4933869568, inactive=1625284608, buffers=398352384, cached=2268397568, shared=390672384, slab=400437248)

In [10]:
def get_embedding_matrix(generate_embedding_matrix=None):
  # Decide if we generate the embedding matrix from from the Google word2vec file,
  # our read it directly from a CSV file we created before
  
  if generate_embedding_matrix is None:
    # check if there is enough RAM available
    available_memory = psutil.virtual_memory().available
    generate_embedding_matrix = (available_memory >= REQUIRED_MEMORY * 2**30)
    
    # check if the Google word2vec file can be opened successfully
    try: 
      with open(EMBEDDING_W2C_FILE) as _: 
        pass
    except:
      print("Could not open the Google word2vec file")
      generate_embedding_matrix = False
    
  # Generate the embedding matrix using either method
  if generate_embedding_matrix:
    print("Generating the embedding matrix from the Google word2vec file, please wait...")
  
    # We need 1 more line for oov (out of vocabulary words)
    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  
    # Load the Google word2vec file. ** This is the RAM-consuming step! **
    word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_W2C_FILE, binary=True)
  
    # Fill the 300 first columns of the matrix from the Google word2vec file.
    # Replace the oov with a random embedding vector.
    oov = 2.0 * np.random.rand(BASE_EMBEDDING_DIM) - 1.0
    oov /= np.linalg.norm(oov)
    for word, i in word_index.items():
      if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
      else:
        embedding_matrix[i] = oov

    # Export the result to CSV using Pandas for future use on a less powerful computer
    pd.DataFrame(embedding_matrix).to_csv(EMBEDDING_MATRIX_FILE, header=False, index=False)
    print("Done.")
        
  else:
    # The CSV file mentioned here was created before as above, but on a more powerful computer
    print("Reading the embedding matrix from a CSV file of us... Done.") 

    # Import the embedding matrix from CSV using Pandas
    embedding_matrix = pd.read_csv(EMBEDDING_MATRIX_FILE, header=None)
    
  return embedding_matrix

In [11]:
embedding_matrix = get_embedding_matrix()

Could not open the Google word2vec file
Reading the embedding matrix from a CSV file of us... Done.


In [12]:
embedding_matrix.shape

(37785, 300)

In [13]:
embedding_matrix.head

<bound method NDFrame.head of             0         1         2         3         4         5         6    \
0      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1     -0.084055  0.055533 -0.012219  0.044333  0.094827  0.007637  0.000222   
2     -0.020020  0.011353  0.188477 -0.182617  0.047363  0.041016  0.017822   
3      0.084961 -0.095215  0.119141  0.111816 -0.111328  0.049805  0.114258   
4      0.059814  0.023804  0.067383  0.316406 -0.189453  0.016479  0.153320   
5     -0.026367  0.068359 -0.031128  0.219727  0.003418 -0.009033  0.107910   
6     -0.150391  0.072266  0.030518  0.041504  0.018066 -0.036865  0.168945   
7      0.033203 -0.089844 -0.294922  0.115234 -0.071289 -0.053955  0.010498   
8     -0.055664  0.008911 -0.092285  0.214844 -0.198242 -0.036133  0.211914   
9      0.015747  0.076660 -0.032227  0.020142  0.015259 -0.052979 -0.014465   
10    -0.291016 -0.026978  0.025757  0.109375 -0.018799 -0.241211 -0.168945   
11    -0.047363  0.187

### Create the dataset

In [14]:
# One-hot vectors
labels_train_3 = to_categorical(np.asarray(sentiments_train_3), 3) 
labels_train_7 = to_categorical(np.asarray(sentiments_train_7), 7)
print(labels_train_3.shape)
print(labels_train_7.shape)

(50333, 3)
(1630, 7)


In [15]:
# Create and split dataset_3
split_idx = int(len(data_train_3) * 0.70)

x_train_3, x_val_3 = data_train_3[:split_idx], data_train_3[split_idx:]
y_train_3, y_val_3 = labels_train_3[:split_idx], labels_train_3[split_idx:]

print('training set: ' + str(len(x_train_3)) + ' samples')
print('validation set: ' + str(len(x_val_3)) + ' samples')

print('x_train_3 shape: ', x_train_3.shape)
print('y_train_3 shape: ', y_train_3.shape)

dataset_3 = x_train_3, y_train_3, x_val_3, y_val_3 

training set: 35233 samples
validation set: 15100 samples
x_train_3 shape:  (35233, 32)
y_train_3 shape:  (35233, 3)


In [16]:
# Create and split dataset_7
split_idx = int(len(data_train_7) * 0.85)

x_train_7, x_val_7 = data_train_7[:split_idx], data_train_7[split_idx:]
y_train_7, y_val_7 = labels_train_7 [:split_idx], labels_train_7[split_idx:]

print('training set: ' + str(len(x_train_7)) + ' samples')
print('validation set: ' + str(len(x_val_7)) + ' samples')

print('x_train_7 shape: ', x_train_7.shape)
print('y_train_7 shape: ', y_train_7.shape)

dataset_7 = x_train_7, y_train_7, x_val_7, y_val_7 

training set: 1385 samples
validation set: 245 samples
x_train_7 shape:  (1385, 32)
y_train_7 shape:  (1385, 7)


## 2. Model training

### A model factory class

In [17]:
class Model:
  
  def build(self, layers, transferred_model):
    # set a new model
    if transferred_model is None: 
      self.model = Sequential()
    else:
      self.model = transferred_model
      
    # add/remove the layers
    for layer in layers:
      if layer[0] == 'Embedding':
        self.model.add(layer[1])
      if layer[0] == 'LSTM':
        self.model.add(LSTM(layer[1]))
      if layer[0] == 'GRU':
        self.model.add(GRU(layer[1]))
      if layer[0] == 'Dropout':
        self.model.add(Dropout(layer[1]))
      if layer[0] == 'Dense':
        self.model.add(Dense(layer[1], activation=layer[2]))
      if layer[0] == 'Cut':
        for _ in range(layer[1]):
          print("Cut")
          self.model.layers.pop()
    
    # compile the model, and show its summary
    self.model.compile(loss=self.hp['loss'],
                  optimizer=self.hp['optimizer'],
                  metrics=self.hp['metrics'])
    self.model.summary()
  
  def __init__(self, dataset, layers, hyperparameters, transferred_model=None):
    self.x_train, self. y_train, self.x_val, self.y_val = dataset
    self.hp = hyperparameters
    self.build(layers, transferred_model) # build the model (set self.model accordingly)
    
  def fit(self):
    self.model.fit(self.x_train, self.y_train, validation_data=(self.x_val, self.y_val),
                        epochs=self.hp['epochs'], batch_size=self.hp['batch_size'])

  def predict(self, data_test):
     self.model.predict(data_test)
    
  def save(self, path):
    self.model.save(path)

### Create the embedding layer

In [18]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False,
                            name='embedding_layer')

### Model training on dataset_3

In [19]:
# Model 1
layers_1 = [
  ('Embedding', embedding_layer),
  ('LSTM', 32),
  ('Dropout', 0.2),
  ('Dense', 32, 'relu'),
  ('Dropout', 0.2),
  ('Dense', 3, 'softmax')
]

hyperparameters_1 = {
  'loss': 'categorical_crossentropy',
  'optimizer': 'Adam', 
  'metrics': ['acc'],
  'epochs': 1, # FIXME
  'batch_size': 50 
}

model1 = Model(dataset_3, layers_1, hyperparameters_1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11335500  
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 11,379,279
Trainable params: 43,779
Non-trainable params: 11,335,500
___________________________________________________________

In [20]:
# Model 2
layers_2 = [
  ('Embedding', embedding_layer),
  ('GRU', 32),
  ('Dropout', 0.2),
  ('Dense', 3, 'softmax')
]

hyperparameters_2 = {
  'loss': 'categorical_crossentropy',
  'optimizer': 'rmsprop', 
  'metrics': ['acc'],
  'epochs': 6,
  'batch_size': 50 
}

model2 = Model(dataset_3, layers_2, hyperparameters_2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11335500  
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                31968     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 11,367,567
Trainable params: 32,067
Non-trainable params: 11,335,500
_________________________________________________________________


In [21]:
# Model call (training + save)
model1.fit()
model1.save("./model1.h5")

Train on 35233 samples, validate on 15100 samples
Epoch 1/1
35233/35233 [==============================] - 20s 569us/step - loss: 0.8419 - acc: 0.6090 - val_loss: 0.8201 - val_acc: 0.6017


### Model training on dataset_7 by transfer learning

In [22]:
# Model built from transfer learning

transferred_model = load_model("./model1.h5") #or: model1.model
transferred_model.summary()

layers = [
  ('Cut', 2),
  ('Dense', 150, 'relu'),
  ('Dense', 64, 'relu'),
  ('Dense', 7, 'softmax'),
]

hyperparameters = {
  'loss': 'categorical_crossentropy',
  'optimizer': 'rmsprop', 
  'metrics': ['acc'],
  'epochs': 11,
  'batch_size': 50
}

model = Model(dataset_7, layers, hyperparameters, transferred_model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11335500  
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 11,379,279
Trainable params: 43,779
Non-trainable params: 11,335,500
___________________________________________________________

In [21]:
model.fit()
model.save("./model3.h5")

Train on 1385 samples, validate on 245 samples
Epoch 1/11
1385/1385 [==============================] - 3s 2ms/step - loss: 1.8253 - acc: 0.2780 - val_loss: 1.7866 - val_acc: 0.2571
Epoch 2/11
1385/1385 [==============================] - 1s 849us/step - loss: 1.7004 - acc: 0.3329 - val_loss: 1.7166 - val_acc: 0.2694
Epoch 3/11
1385/1385 [==============================] - 1s 869us/step - loss: 1.6408 - acc: 0.3466 - val_loss: 1.7067 - val_acc: 0.2776
Epoch 4/11
1385/1385 [==============================] - 1s 859us/step - loss: 1.6067 - acc: 0.3466 - val_loss: 1.7022 - val_acc: 0.2735
Epoch 5/11
1385/1385 [==============================] - 1s 866us/step - loss: 1.5878 - acc: 0.3523 - val_loss: 1.6929 - val_acc: 0.2776
Epoch 6/11
1385/1385 [==============================] - 1s 882us/step - loss: 1.5578 - acc: 0.3639 - val_loss: 1.6797 - val_acc: 0.2735
Epoch 7/11
1385/1385 [==============================] - 1s 874us/step - loss: 1.5459 - acc: 0.3755 - val_loss: 1.6768 - val_acc: 0.3429
Epo

## 3. Model testing

In [22]:
r = model.predict(data_test_7)

conclusions = ["-1: slightly negative emotional state can be inferred",
               "-2: moderately negative emotional state can be inferred",
               "-3: very negative emotional state can be inferred",
               "0: neutral or mixed emotional state can be inferred",
               "1: slightly positive emotional state can be inferred",
               "2: moderately positive emotional state can be inferred",
               "3: very positive emotional state can be inferred"]

for i in range(len(r)):
    data['Intensity Class'][i] = conclusions[numpy.argmax(r[i])]

TypeError: object of type 'NoneType' has no len()

### Write predictions

In [ ]:
#FIXME: write predictions with Pandas instead

with open("./submission_after.csv", "w") as f:
  f.write("ID\tTweet\tAffect Dimension\tIntensity Class\n")
  for d in range(len(data)):
    f.write(data['ID'][d] + "\t" + data['Tweet'][d] + "\tvalence\t" + data['Intensity Class'][d] + "\n")

In [2]:
model = load_model("./model1.h5")

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11335500  
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 99        
Total params: 11,379,279
Trainable params: 43,779
Non-trainable params: 11,335,500
___________________________________________________________

In [3]:
model.layers.pop()
model.layers.pop()

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11335500  
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 99        
Total params: 11,379,279
Trainable params: 43,779
Non-trainable params: 11,335,500
___________________________________________________________

In [5]:
model.outputs = [model.layers[-1].output]
model.layers[-1].outbound_nodes = []

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11335500  
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 99        
Total params: 11,379,279
Trainable params: 43,779
Non-trainable params: 11,335,500
___________________________________________________________

In [10]:
model.compile(optimizer='Adam', loss='categorical_crossentropy')

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11335500  
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 99        
Total params: 11,379,279
Trainable params: 43,779
Non-trainable params: 11,335,500
___________________________________________________________